In [2]:
import torch
from torchvision import datasets,transforms


In [3]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)



  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting /home/mohamed/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/mohamed/.pytorch/F_MNIST_data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting /home/mohamed/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home/mohamed/.pytorch/F_MNIST_data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting /home/mohamed/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/mohamed/.pytorch/F_MNIST_data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting /home/mohamed/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/mohamed/.pytorch/F_MNIST_data/FashionMNIST/raw



In [4]:
from torch import nn,optim
import torch.nn.functional as f

In [11]:
class classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = f.relu(self.fc3(x))
        x = f.log_softmax(self.fc4(x), dim=1)
        
        return x
    

Now you should create your network and train it. First you'll want to define the criterion (something like nn.CrossEntropyLoss or nn.NLLLoss) and the optimizer (typically optim.SGD or optim.Adam).

In [12]:
model=classifier()
criterion=nn.NLLLoss()
optimizer=optim.Adam(model.parameters(),lr=.001)


In [16]:
epoch=10
for i in range(10):
    lose=0
    for im,la in trainloader:
        log_p=model(im)
        loss=criterion(log_p,la)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lose+=loss.item()
    else:
        print(f"Training loss: {lose/len(trainloader)}")

Training loss: 0.37574160013244606
Training loss: 0.3366376662622891
Training loss: 0.3121396289117682
Training loss: 0.2911254740647797
Training loss: 0.27483169110154293
Training loss: 0.26433746637438915
Training loss: 0.2502300455086013
Training loss: 0.23867582044462915
Training loss: 0.2276772339063794
Training loss: 0.21719923532053606


In [18]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# Test out your network!

dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[1]

# TODO: Calculate the class probabilities (softmax) for img
ps = torch.exp(model(img))
ps


tensor([[4.2362e-02, 1.7374e-04, 7.1703e-01, 8.9311e-03, 7.9782e-02, 7.3554e-07,
         1.5162e-01, 9.1045e-08, 9.3609e-05, 1.5301e-06]],
       grad_fn=<ExpBackward0>)

In [19]:
top_p, top_class = ps.topk(1, dim=1)
# Look at the most likely classes for the first 10 examples
print(top_class[:10,:])

tensor([[2]])


In [22]:


model = classifier()
criterion = nn.NLLLoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30

train_losses, test_losses = [], []
for e in range(epochs):
    tot_train_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        tot_train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    else:
        tot_test_loss = 0
        test_correct = 0  # Number of correct predictions on the test set
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for images, labels in testloader:
                log_ps = model(images)
                loss = criterion(log_ps, labels)
                tot_test_loss += loss.item()

                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                test_correct += equals.sum().item()

        # Get mean loss to enable comparison between train and test sets
        train_loss = tot_train_loss / len(trainloader.dataset)
        test_loss = tot_test_loss / len(testloader.dataset)

        # At completion of epoch
        train_losses.append(train_loss)
        test_losses.append(test_loss)

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_loss),
              "Test Loss: {:.3f}.. ".format(test_loss),
              "Test Accuracy: {:.3f}".format(test_correct / len(testloader.dataset)))



Epoch: 1/30..  Training Loss: 0.526..  Test Loss: 0.451..  Test Accuracy: 0.832
Epoch: 2/30..  Training Loss: 0.393..  Test Loss: 0.453..  Test Accuracy: 0.838
Epoch: 3/30..  Training Loss: 0.357..  Test Loss: 0.392..  Test Accuracy: 0.854
Epoch: 4/30..  Training Loss: 0.333..  Test Loss: 0.380..  Test Accuracy: 0.862
Epoch: 5/30..  Training Loss: 0.316..  Test Loss: 0.366..  Test Accuracy: 0.869
Epoch: 6/30..  Training Loss: 0.302..  Test Loss: 0.355..  Test Accuracy: 0.873
Epoch: 7/30..  Training Loss: 0.290..  Test Loss: 0.353..  Test Accuracy: 0.877
Epoch: 8/30..  Training Loss: 0.281..  Test Loss: 0.373..  Test Accuracy: 0.873
Epoch: 9/30..  Training Loss: 0.273..  Test Loss: 0.378..  Test Accuracy: 0.869
Epoch: 10/30..  Training Loss: 0.263..  Test Loss: 0.370..  Test Accuracy: 0.878
Epoch: 11/30..  Training Loss: 0.258..  Test Loss: 0.369..  Test Accuracy: 0.875
Epoch: 12/30..  Training Loss: 0.254..  Test Loss: 0.388..  Test Accuracy: 0.875
Epoch: 13/30..  Training Loss: 0.245.

In [ ]:


plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)



In [ ]:

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)

        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        # Now with dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))

        # output so no dropout here
        x = F.log_softmax(self.fc4(x), dim=1)

        return x
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            model.eval()
            for images, labels in testloader:
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train()
        
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Test Loss: {:.3f}.. ".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)

In [ ]:
#inference 
model.eval()

dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[0]
# Convert 2D image to 1D vector
img = img.view(1, 784)

# Calculate the class probabilities (softmax) for img
with torch.no_grad():
    output = model.forward(img)

ps = torch.exp(output)